# Environment setup

In [ ]:
# modify with caution, the change here may cause error
%cd
%cd ..
%cd content
%mkdir pupae_detector
%cd pupae_detector
!git clone https://github.com/ultralytics/yolov5  # clone
%cd /content/pupae_detector/yolov5

%pip install -qr requirements.txt  # install
%pip install roboflow
%pip install opencv-python==4.5.5.64

from roboflow import Roboflow
import torch
import utils
import os
display = utils.notebook_init()  # checks

YOLOv5 🚀 v6.2-196-ge42c89d Python-3.7.14 torch-1.12.1+cu113 CPU


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 38.7/107.7 GB disk)


In [22]:
from google.colab import drive
drive.mount('/content/drive')
dir = "/content/drive/MyDrive/pupae_detector_folder" 
if not os.path.exists(dir):
  os.mkdir(dir)

# %cp -R /content/pupae_detector/yolov5/runs  /content/drive/MyDrive/pupae_detector_folder/
%cp -R /content/drive/MyDrive/pupae_detector_folder/runs /content/pupae_detector/yolov5 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Place to put your dataset code here

In [ ]:
os.environ["DATASET_DIRECTORY"] = "/content/pupae_detector/datasets"


# replace here with your own dataset
rf = Roboflow(api_key="N6BLYmALrZnv9R94uWiQ")
project = rf.workspace("university-of-adelaide-wt6zr").project("10_10")
dataset = project.version(1).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to /content/pupae_detector/datasets/10_10-1 in yolov5pytorch:: 100%|██████████| 60/60 [00:00<00:00, 1055.91it/s]


# Training section


In [23]:
%cd /content/pupae_detector/yolov5


!python train.py --img 512 --batch 32 --epochs 5 --data {dataset.location}/data.yaml --weights yolov5s.pt --cache
%cp -R /content/pupae_detector/yolov5/runs/train/  /content/drive/MyDrive/pupae_detector_folder/runs/train/

/content/pupae_detector/yolov5
train: weights=yolov5s.pt, cfg=, data=/content/pupae_detector/datasets/10_10-1/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=5, batch_size=32, imgsz=512, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-196-ge42c89d Python-3.7.14 torch-1.12.1+cu113 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_ga

# Detection section


In [ ]:
# modify here

#version of model, if you dont have any recon version, use 0 for the latest model
model = 0
#change here for different style of detection box
only_show_box = False
#if you want to save each detection, change here to True
back_up = True

#detection parameter
img_size = 2024
confidence = 0.4
iou_value = 0.3


%cd /content/pupae_detector/yolov5

# detect latest model version
import fnmatch
if model == 0  and back_up == True:
  dir_path = r'/content/drive/MyDrive/pupae_detector_folder/runs/train'
  # dir_path = r'/content/pupae_detector/yolov5/runs/train'
  model = len(fnmatch.filter(os.listdir(dir_path), '*'))



if model is 1:
  model_name = ""
else:
  model_name = str(model)

print("Using training model:",model_name)

# target = "runs/train/exp"+model_name+"/weights/best.pt"
target = "/content/drive/MyDrive/pupae_detector_folder/runs/train/exp"+model_name+"/weights/best.pt"

if only_show_box is True:
  box_setting = "--hide-labels --hide-conf"
else:
  box_setting = ""

if back_up is True:
  back = ""
else:
  back = "--exist-ok"
  %cd runs/detect/exp/
  %rm -rf labels
  %cd /content/pupae_detector/yolov5

location = dataset.location+"/test/images"

# change your number here for better detection
!python detect.py --weights {target} --img {img_size} --conf {confidence} --iou {iou_value} --source {location} {box_setting} {back} --save-txt  --line-thickness 1

%cp -R /content/pupae_detector/yolov5/runs/detect/  /content/drive/MyDrive/pupae_detector_folder/runs/detect/

/content/pupae_detector/yolov5
Using training model: 
detect: weights=['/content/drive/MyDrive/pupae_detector_folder/runs/train/exp/weights/best.pt'], source=/content/pupae_detector/datasets/10_10-1/test/images, data=data/coco128.yaml, imgsz=[2024, 2024], conf_thres=0.4, iou_thres=0.3, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=1, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-196-ge42c89d Python-3.7.14 torch-1.12.1+cu113 CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
WARNING ⚠️ --img-size [2024, 2024] must be multiple of max stride 32, updating to [2048, 2048]
image 1/4 /content/pupae_detector/datasets/10_10-1/test/images/IMG_20221010_131358_jpg.rf.ed40ef06ec10f663b1acb1f18b92048b.jpg: 1536x2048 10 pup

# Show result

In [ ]:
#if you change back_up above to true, it will automatically show the latest detect result, if you want to view specific result, change here.
exp_version = 0

# detect latest model version
if exp_version == 0  and back_up == True:
  dir_path = r'/content/pupae_detector/yolov5/runs/detect'
  exp_version = len(fnmatch.filter(os.listdir(dir_path), '*'))

if back_up is False:
  exp_version = 1
  
if exp_version is 1:
  exp_name = ""
else:
  exp_name = str(exp_version)

print("Using detection result:",exp_name)

import glob
from pathlib import Path
from IPython.display import Image, display

# %cd /content/pupae_detector/yolov5
for imageName in glob.glob('/content/pupae_detector/yolov5/runs/detect/exp'+exp_name+'/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")
    
print("Detect version : exp"+exp_name)
for txt in glob.glob('/content/pupae_detector/yolov5/runs/detect/exp'+exp_name+'/labels/*.txt'): #assuming TXT
    Num = 0
    fp = open(txt, "r")
    Num = len(fp.readlines())
    print("Number of pupae in image ["+ Path(txt).stem +"]: \n", Num)


# File management code


<p>
In this toggle, every line start with #(comments), which means it's not in funtion in this step. You can delete the # in front to make that line of code functional and recomment it by adding # back

all the code start with %
</p>
<h3>Code syntax</h3>
<p># Code1 usage  </p>
<p># function explaination</p>
<p>Code1_code</p>
<p>Code1_code</p>
<br>
<p># Code2 usage</p>
<p>...</p>

In [ ]:
#1.Delete all local dataset
#This is use to clean up all the datasets you import and use before 
# %cd /content/pupae_detector/
# %rm -rf datasets

#2.Delete all models 
#This is use to clean up all detect models, after using this code, your next model start from 1
# %cd /content/pupae_detector/yolov5/runs/
# %rm -rf train

#3.Delete all experiments 
#This is use to clean up all detect experiments, after using this code, your next experiment start from 1
# %cd /content/pupae_detector/yolov5/runs/
# %rm -rf detect

%cp -R /content/pupae_detector/yolov5/runs  /content/drive/MyDrive/pupae_detector_folder/

#4.Delete pupae detector
#This will delete all the information including code from your local
# %cd /content/
# %rm -rf pupae_detector


#save info to google_drive
# scp -r ~/content.pupae_detector/yolov5/runs ~/content/drive/pupae_counter 

In [21]:
%rm -rf /content/pupae_detector/yolov5/pupae_detector_folder/